In [1]:
workspace_id = "<your-workspace-id>"
workspace_token = "<your-workspace-token>"

In [2]:
!pip install -U scikit-learn

    100% |ââââââââââââââââââââââââââââââââ| 5.5MB 58kB/s  eta 0:00:01
    100% |ââââââââââââââââââââââââââââââââ| 13.8MB 28kB/s  eta 0:00:01
    100% |ââââââââââââââââââââââââââââââââ| 30.8MB 14kB/s eta 0:00:011
  Found existing installation: numpy 1.13.3
    Uninstalling numpy-1.13.3:
      Successfully uninstalled numpy-1.13.3
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
  Found existing installation: scikit-learn 0.15.1
    Uninstalling scikit-learn-0.15.1:
      Successfully uninstalled scikit-learn-0.15.1
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics


dataset_url = "https://raw.githubusercontent.com/apogiatzis/breast-cancer-azure-ml-notebook/master/breast-cancer-data.csv"
df = pd.read_csv(dataset_url)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

In [4]:
df['diagnosis']=df['diagnosis'].map({'M':1,'B':0})

train, test = train_test_split(df, test_size = 0.2)

features = ['texture_mean','perimeter_mean','smoothness_mean','compactness_mean','symmetry_mean']

train_X = train[features]
train_y=train.diagnosis

test_X= test[features]
test_y =test.diagnosis

print(train.shape)
print(test.shape)

train_X.head()

(455, 33)
(114, 33)


,texture_mean,perimeter_mean,smoothness_mean,compactness_mean,symmetry_mean
43,20.28,87.32,0.1041,0.1436,0.1974
282,18.18,127.20,0.1037,0.1442,0.1893
389,23.21,128.90,0.1010,0.1318,0.1989
148,15.18,93.97,0.0997,0.1021,0.1724
400,21.02,124.40,0.1230,0.2576,0.2113


In [5]:
model=RandomForestClassifier(n_estimators=100, n_jobs=-1)
model.fit(train_X,train_y)

prediction=model.predict(test_X)
metrics.accuracy_score(prediction,test_y)

0.9035087719298246

In [6]:
from azureml import services
@services.publish(workspace_id, workspace_token)
@services.types(texture_mean=float, perimeter_mean=float, smoothness_mean=float, compactness_mean=float, symmetry_mean=float)
@services.returns(int)

def breast_cancer_predictor(texture_mean, perimeter_mean, smoothness_mean, compactness_mean, symmetry_mean):
    return model.predict(np.array([[texture_mean, perimeter_mean, smoothness_mean, compactness_mean, symmetry_mean]]))

# hold onto information about your web service so you can call it within the notebook later
service_url = breast_cancer_predictor.service.url 
api_key = breast_cancer_predictor.service.api_key
help_url = breast_cancer_predictor.service.help_url
service_id = breast_cancer_predictor.service.service_id

In [7]:
breast_cancer_predictor.service(018.14, 75.00, 0.09968, 0.05914, 0.1619)

1

In [12]:
import urllib2
import json

data = {"Inputs": {"input1": { "ColumnNames": [ "texture_mean", "perimeter_mean", "smoothness_mean", "compactness_mean", "symmetry_mean"],
                               "Values": [["018.14", "75.00", "0.09968", "0.05914", "0.1619"]]}}, # specified feature values
        "GlobalParameters": {} }
body = json.dumps(data)
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib2.Request(service_url, body, headers) 
try:
    response = urllib2.urlopen(req)
    result = json.loads(response.read())  
    print(result['Results']['output1']['value']['Values'][0][0]) # get the returned prediction
except urllib2.HTTPError, error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(json.loads(error.read())) 

1
